<b>Evaluation Metrics</b>

In [ ]:
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error
import numpy as np
from PIL import Image

def resize_image(image, target_size=(512, 512)):
    # Resize the image to the target size
    return image.resize(target_size)

def preprocess_image(image):
    # Convert to grayscale
    gray_image = np.array(image.convert("L"))
    return gray_image

def calculate_metrics(original_folder, registered_folder):
    # Get a list of all files in the original and registered folders
    original_files = os.listdir(original_folder)
    registered_files = os.listdir(registered_folder)

    # Initialize empty lists to store metric values
    ssim_values = []
    mse_values = []
    ncc_values = []

    # Iterate over the files and calculate metrics for each pair
    for original_file in original_files:
        if original_file in registered_files:
            # Construct the file paths
            original_image_path = os.path.join(original_folder, original_file)
            registered_image_path = os.path.join(registered_folder, original_file)

            # Read and resize the images using the PIL library
            original_image = Image.open(original_image_path)
            registered_image = Image.open(registered_image_path)

            # Resize images to a common size of (512, 512)
            original_image = resize_image(original_image)
            registered_image = resize_image(registered_image)

            # Convert the images to NumPy arrays
            original_image_np = preprocess_image(original_image)
            registered_image_np = preprocess_image(registered_image)

            # Calculate SSIM
            ssim_score = ssim(original_image_np, registered_image_np, multichannel=False)

            # Calculate MSE
            mse = mean_squared_error(original_image_np.flatten(), registered_image_np.flatten())

            # Calculate NCC
            ncc = np.corrcoef(original_image_np.flatten(), registered_image_np.flatten())[0, 1]

            ssim_values.append(ssim_score)
            mse_values.append(mse)
            ncc_values.append(ncc)

    # Convert the lists to NumPy arrays for easier calculations
    ssim_values = np.array(ssim_values)
    mse_values = np.array(mse_values)
    ncc_values = np.array(ncc_values)

    # Calculate mean and standard deviation of the metrics
    mean_ssim = np.mean(ssim_values)
    std_ssim = np.std(ssim_values)
    mean_mse = np.mean(mse_values)
    std_mse = np.std(mse_values)
    mean_ncc = np.mean(ncc_values)
    std_ncc = np.std(ncc_values)

    return mean_ssim, std_ssim, mean_mse, std_mse, mean_ncc, std_ncc

# Example usage
original_folder = '/home/souraja/DFMIR/Dataset/testBdataset2_resized'
registered_folder = '/home/souraja/transmorph/TransMorph/Dataset/test_data/registered'

# Calculate the mean and standard deviation of SSIM, MSE, and NCC
mean_ssim, std_ssim, mean_mse, std_mse, mean_ncc, std_ncc = calculate_metrics(original_folder, registered_folder)

# Print the results
print(f"Mean SSIM: {mean_ssim:.4f}")
print(f"Standard Deviation of SSIM: {std_ssim:.4f}")
print(f"Mean MSE: {mean_mse:.2f}")
print(f"Standard Deviation of MSE: {std_mse:.2f}")
print(f"Mean NCC: {mean_ncc:.4f}")
print(f"Standard Deviation of NCC: {std_ncc:.4f}")



<b>Loss vs Epoch Plot from log file</b>

In [ ]:
import re
import matplotlib.pyplot as plt

def parse_loss_log(log_file):
    epoch_numbers = []
    loss_values = {key: [] for key in ['G', 'NCE', 'R', 'smooth', 'local', 'NCE_Y']}

    with open(log_file, 'r') as file:
        for line in file:
            # Check if the line contains loss values
            if re.search(r'G: (\d+\.\d+) NCE: (\d+\.\d+) R: (\d+\.\d+) smooth: (\d+\.\d+) local: (\d+\.\d+) NCE_Y: (\d+\.\d+)', line):
                epoch_match = re.search(r'epoch: (\d+)', line)
                if epoch_match:
                    epoch_number = int(epoch_match.group(1))
                    epoch_numbers.append(epoch_number)

                    # Extract loss values
                    loss_match = re.search(r'G: (\d+\.\d+) NCE: (\d+\.\d+) R: (\d+\.\d+) smooth: (\d+\.\d+) local: (\d+\.\d+) NCE_Y: (\d+\.\d+)', line)
                    if loss_match:
                        for key, value in zip(['G', 'NCE', 'R', 'smooth', 'local', 'NCE_Y'], map(float, loss_match.groups())):
                            loss_values[key].append(value)

    return epoch_numbers, loss_values

def plot_loss_graph(loss_logs, labels):
    plt.figure(figsize=(12, 6))

    for i, loss_log in enumerate(loss_logs):
        epoch_numbers, loss_values = parse_loss_log(loss_log)

        for key in loss_values:
            plt.plot(epoch_numbers, loss_values[key], label=f'{labels[i]} - {key}')

    plt.xlabel('Epoch Number')
    plt.ylabel('Loss Values')
    plt.title('Loss Values vs. Epochs')
    plt.legend()
    plt.show()

# Provide the paths to the loss log files
loss_logs = [
    'srcpath/checkpoints/experiment_name/loss_log.txt'
]

# Provide the labels for the loss logs
labels = [
    'Experiment 1'
]

plot_loss_graph(loss_logs, labels)


<b>Calculating DSC in ROI </b>
(ROI defined by enclosing the lymph nodes in both the MRI images by the smallest rectangle)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def draw_box(image_path, box_coordinates, title):
    # Open the image using PIL
    image = Image.open(image_path)

    # Create figure and axis
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image, cmap='gray')

    # Extract box coordinates
    x, y, width, height = box_coordinates

    # Create a rectangle patch
    rect = patches.Rectangle((x, y), width, height, linewidth=0.5, edgecolor='red', facecolor='none')

    # Add the rectangle to the axis
    ax.add_patch(rect)

    # Set axis labels and title
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_title(title)

    # Show the plot
    plt.show()

def calculate_dsc(box1, box2):
    # Calculate Dice Similarity Coefficient for the two boxes
    intersection_x = max(0, min(box1[0] + box1[2], box2[0] + box2[2]) - max(box1[0], box2[0]))
    intersection_y = max(0, min(box1[1] + box1[3], box2[1] + box2[3]) - max(box1[1], box2[1]))
    intersection_area = intersection_x * intersection_y

    union_area = (box1[2] * box1[3]) + (box2[2] * box2[3]) - intersection_area

    dsc = (intersection_area) / union_area if union_area != 0 else 0.0
    return dsc * 100  # Multiply by 100 to get percentage

# File addresses of the original and generated images
original_image_path = 'srcpath/Dataset/origMRI.jpg'
generated_image_path = 'srcpath/results/finalMRI.jpg'

# Coordinates of the box for the original image (x, y, width, height)
box_coordinates_original = (260, 321, 15, 25)  # Example coordinates, replace with your actual values

# Coordinates of the box for the generated image (x, y, width, height)
box_coordinates_generated = (260, 324, 15, 25)  # Example coordinates, replace with your actual values

# Draw the box on the original image
draw_box(original_image_path, box_coordinates_original, title='Original MRI with Box')

# Draw the box on the generated image
draw_box(generated_image_path, box_coordinates_generated, title='Generated MRI with Box')

# Calculate DSC for the two boxes
dsc_percentage = calculate_dsc(box_coordinates_original, box_coordinates_generated)
print(f'DSC Percentage for ROI: {dsc_percentage:.2f}%')
